# Impurity Measures on Mushroom Dataset

Σε αυτό το notebook θα δούμε πώς **διαφορετικές μετρικές καθαρότητας** (impurity measures)
επηρεάζουν:

- την **αξιολόγηση των χαρακτηριστικών** (ποια θεωρούνται πιο σημαντικά)
- και έμμεσα τις **επιλογές ενός δέντρου αποφάσεων**.

Θα χρησιμοποιήσουμε το *Mushroom Classification* dataset:

- Στόχος: `class` (edible (βρώσιμο) `e` ή poisonous (δηλητηριώδες) `p`)
- Χαρακτηριστικά: αποκλειστικά **κατηγορικά** (π.χ. `odor`, `cap-color`, `gill-size`)

Μετρικές που θα υπολογίσουμε για κάθε χαρακτηριστικό:

- **Entropy / Information Gain**
- **Split Information & Gain Ratio**
- **Gini Impurity / Gini Gain**

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

# Αν το notebook βρίσκεται στον φάκελο notebooks/, τότε το ROOT είναι ένας κατάλογος πάνω
ROOT = Path("..").resolve()
DATA_PATH = ROOT / "data" / "mushrooms.csv"

In [ ]:
# ----------------------------------------------
# ΥΠΟΛΟΓΙΣΜΟΣ ENTROPY
# ----------------------------------------------
def entropy(y: pd.Series) -> float:
    """Υπολογίζει την εντροπία (entropy) για μια στήλη κατηγορικών τιμών.

    Τύπος:
        H = - Σ (p_i * log2(p_i))

    όπου p_i είναι η σχετική συχνότητα κάθε κατηγορίας.
    Αν όλες οι τιμές είναι ίδιες -> H = 0 (τέλεια καθαρότητα)
    Αν οι τιμές είναι ισοκατανεμημένες -> H = log2(k) (μέγιστη αβεβαιότητα)
    """
    p = y.value_counts(normalize=True)
    return float(-(p * np.log2(p)).sum())


# ----------------------------------------------
# ΥΠΟΛΟΓΙΣΜΟΣ GINI IMPURITY
# ----------------------------------------------
def gini_impurity(y: pd.Series) -> float:
    """Υπολογίζει την αβεβαιότητα Gini για μια στήλη κατηγορικών τιμών.

    Τύπος:
        G = 1 - Σ (p_i)^2

    Όπου p_i η πιθανότητα κάθε κατηγορίας.
    Όσο μικρότερη η G, τόσο πιο "καθαρός" ο κόμβος.
    """
    p = y.value_counts(normalize=True)
    return float(1.0 - (p**2).sum())


# ----------------------------------------------
# ΥΠΟΛΟΓΙΣΜΟΣ ΠΛΗΡΟΦΟΡΙΑΣ ΚΑΙ ΔΕΙΚΤΩΝ ΓΙΑ ΕΝΑ ΧΑΡΑΚΤΗΡΙΣΤΙΚΟ
# ----------------------------------------------
def feature_scores(df: pd.DataFrame, feature: str, target: str):
    """
    Υπολογίζει 4 δείκτες καθαρότητας για ένα κατηγορικό χαρακτηριστικό:

      - Information Gain (IG)
      - Split Information
      - Gain Ratio
      - Gini Gain

    Οι δείκτες αυτοί δείχνουν πόσο "πληροφοριακό" είναι το χαρακτηριστικό
    για την πρόβλεψη της μεταβλητής-στόχου.
    """

    y = df[target]
    n = len(df)

    # Πριν το split
    H_before = entropy(y)
    G_before = gini_impurity(y)

    # Μετά το split (weighted)
    H_after = 0.0
    G_after = 0.0
    split_info = 0.0

    # Για κάθε διαφορετική τιμή του feature
    for v, subset in df.groupby(feature):
        weight = len(subset) / n
        y_sub = subset[target]

        H_after += weight * entropy(y_sub)
        G_after += weight * gini_impurity(y_sub)

        if weight > 0:
            split_info -= weight * np.log2(weight)

    info_gain = H_before - H_after
    gini_gain = G_before - G_after
    gain_ratio = info_gain / split_info if split_info > 0 else 0.0

    return info_gain, split_info, gain_ratio, gini_gain

## 1. Φόρτωση δεδομένων & βασική εξερεύνηση

Θα φορτώσουμε το `mushrooms.csv`, θα δούμε τις πρώτες γραμμές και την κατανομή της κλάσης
(edible vs poisonous).

In [ ]:
df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
# Minimal, beginner-friendly dataset overview (pretty-printed tables)
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print('Shape:', df.shape)
display(df.head())
display(df.describe(include='all').T)
display(df.isnull().sum().to_frame('missing'))
display(df['class'].value_counts().to_frame('count'))
display(df['class'].value_counts(normalize=True).to_frame('ratio'))

In [ ]:
# Κατανομή της κλάσης-στόχου (edible vs poisonous)
display(df['class'].value_counts(normalize=True).to_frame('ratio'))

## 2. Υπολογισμός Information Gain, Gain Ratio και Gini Gain

Για κάθε χαρακτηριστικό θα υπολογίσουμε:

- **Information Gain**: πόσο μειώνεται η εντροπία αν κάνουμε split σε αυτό το feature.
- **Split Information**: πόσο "διασκορπισμένες" είναι οι τιμές του feature.
- **Gain Ratio**: IG κανονικοποιημένο με βάση το split info (αντισταθμίζει features με πολλές κατηγορίες).
- **Gini Gain**: αντίστοιχο του IG αλλά με Gini impurity αντί για entropy.

Στόχος: να συγκρίνουμε τη **σειρά σημαντικότητας** των χαρακτηριστικών ανά μετρική.


In [ ]:
target = "class"  # 'e' ή 'p'
features = [c for c in df.columns if c != target]

rows = []
for feat in features:
    ig, si, gr, gg = feature_scores(df, feat, target)
    rows.append(
        {
            "feature": feat,
            "num_values": df[feat].nunique(),
            "info_gain": ig,
            "split_info": si,
            "gain_ratio": gr,
            "gini_gain": gg,
        }
    )

scores = pd.DataFrame(rows)
scores.head()

In [ ]:
print("=== Top 10 features by Information Gain ===")
display(scores.sort_values("info_gain", ascending=False).head(10))

print("\n=== Top 10 features by Gain Ratio ===")
display(scores.sort_values("gain_ratio", ascending=False).head(10))

print("\n=== Top 10 features by Gini Gain ===")
display(scores.sort_values("gini_gain", ascending=False).head(10))

### Παρατήρηση

Συχνά θα δούμε ότι:

- Χαρακτηριστικά όπως η **οσμή (`odor`)** έχουν πολύ υψηλό **Information Gain** και **Gini Gain**,
  γιατί διαχωρίζουν σχεδόν τέλεια φαγώσιμα από δηλητηριώδη μανιτάρια.
- Το **Gain Ratio** μπορεί να αλλάξει ελαφρώς τη σειρά, επειδή τιμωρεί features με
  πάρα πολλές κατηγορίες (ψηλό split info).

Έτσι, διαφορετικές μετρικές μπορεί να προτείνουν **διαφορετική "προτεραιότητα" χαρακτηριστικών**,  
ειδικά όταν κάποια έχουν πολλές μοναδικές τιμές.


## 3. Δέντρα αποφάσεων με διαφορετικά κριτήρια (entropy vs gini)

Τώρα θα εκπαιδεύσουμε δύο δέντρα αποφάσεων:

- Ένα με **criterion = "entropy"**
- Ένα με **criterion = "gini"**

και θα συγκρίνουμε:

- την απόδοση (accuracy),
- τις **feature importances**.

Έτσι θα δούμε στην πράξη πώς οι διαφορετικές μετρικές impurity
επηρεάζουν το ποια χαρακτηριστικά θεωρούνται πιο σημαντικά από το μοντέλο.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Όλα τα features είναι κατηγορικά -> τα κωδικοποιούμε με one-hot
X = pd.get_dummies(df.drop(columns=[target]))
y = (df[target] == 'p').astype(int)  # 1 = poisonous, 0 = edible

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

def train_tree(criterion):
    clf = DecisionTreeClassifier(criterion=criterion, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return clf, acc

tree_entropy, acc_entropy = train_tree("entropy")
tree_gini, acc_gini = train_tree("gini")

print(f"Accuracy (entropy): {acc_entropy:.3f}")
print(f"Accuracy (gini):    {acc_gini:.3f}")

In [ ]:
# Συγκρίνουμε τις 10 πιο σημαντικές μεταβλητές για κάθε κριτήριο
fi_entropy = pd.Series(tree_entropy.feature_importances_, index=X.columns)
fi_gini = pd.Series(tree_gini.feature_importances_, index=X.columns)

# Create DataFrames for pretty print
top_entropy = fi_entropy.sort_values(ascending=False).head(10).to_frame('importance')
top_gini = fi_gini.sort_values(ascending=False).head(10).to_frame('importance')

print("=== Top 10 features (entropy) ===")
display(top_entropy)

print("\n=== Top 10 features (gini) ===")
display(top_gini)

### Συμπεράσματα

- Και τα δύο κριτήρια (**entropy** και **gini**) τείνουν να διαλέγουν
  τα **ίδια πολύ δυνατά χαρακτηριστικά** (π.χ. `odor_*`).
- Οι **ακριβείς σημαντικότητες** (feature importances) και η σειρά κατάταξης
  μπορεί να διαφέρουν λίγο, επειδή η κάθε μετρική "μετράει" την ακαθαρσία
  με διαφορετικό τρόπο.
- Σε δεδομένα σαν τα μανιτάρια, όπου κάποια features είναι *πολύ ξεκάθαρα*,
  οι διαφορές μεταξύ entropy/gini στην πράξη είναι μικρές.

Παρόλα αυτά, η θεωρητική διαφορά στις μετρικές (Information Gain, Gain Ratio, Gini)
παίζει ρόλο στην επιλογή splits, ειδικά όταν υπάρχουν πολλαπλά features
με παρόμοια discriminative δύναμη.


## 📈 Οπτικοποίηση με matplotlib των μετρικών ανά χαρακτηριστικό

In [ ]:
# Bar plot: Top 10 χαρακτηριστικά βάσει Information Gain
top_ig = scores.sort_values("info_gain", ascending=False).head(10)

plt.figure(figsize=(6, 4))
plt.barh(top_ig["feature"], top_ig["info_gain"])
plt.xlabel("Information Gain")
plt.title("Top 10 features by Information Gain")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Bar plot: Top 10 χαρακτηριστικά βάσει Gain Ratio
top_gr = scores.sort_values("gain_ratio", ascending=False).head(10)

plt.figure(figsize=(6, 4))
plt.barh(top_gr["feature"], top_gr["gain_ratio"])
plt.xlabel("Gain Ratio")
plt.title("Top 10 features by Gain Ratio")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Bar plot: Top 10 χαρακτηριστικά βάσει Gini Gain
top_gg = scores.sort_values("gini_gain", ascending=False).head(10)

plt.figure(figsize=(6, 4))
plt.barh(top_gg["feature"], top_gg["gini_gain"])
plt.xlabel("Gini Gain")
plt.title("Top 10 features by Gini Gain")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()